In [1]:
import pandas as pd
from sqlalchemy import create_engine
# In GitBash:
# $ pip install geocoder
import geocoder
import numpy as np
from includes import (gmaps_api_key)
%matplotlib inline
error_count = 0

In [2]:
#create function for geo coding addresses
def geomapper(address):
    #try:
        g = geocoder.google(address, key=gmaps_api_key)
        lat = g.json['lat']
        long = g.json['lng']
        return (f'{lat},{long}')
    #except:
        #error_count += 1
        #return (f'32.7778,-117.2265')

In [3]:
#Create function called locateAddresses that adds latitude and longitude columns for geomapping
#Assumes df have column "Address"

def locateAddresses(df):
    df["geo"] = 0
    df["geo"]= df.Address.apply(lambda x: geomapper(x))
    
    #split geo column to assign values in latitude and longitude 
    df[['Lat', 'Lng']] = df.geo.str.split(',', expand = True)
    #df = df.drop(columns=['geo'])
    del df['geo']
    return

In [4]:
#File 1: Events: SD_Events.csv

In [5]:
#Process File 1
SD_events_file = "Resources/SD_Events.csv"
SD_events_df = pd.read_csv(SD_events_file)
SD_events_df.head()

,Event,Description,Neighborhood,Date,Cost
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,1/20/2019,$$
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,3/16/2019,$
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,5/12/2019,$$
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,3/10/2019,$$
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,4/21/2019,$$


In [6]:
#Convert '$$' in Cost column to numeric
# get the length of the string of column in a dataframe
#http://www.datasciencemadesimple.com/get-string-length-column-dataframe-python-pandas/
#df['Quarters_length'] = df['Quarters'].apply(len)

SD_events_df.Cost = SD_events_df.Cost.apply(len)
SD_events_df.head()

,Event,Description,Neighborhood,Date,Cost
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,1/20/2019,2
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,3/16/2019,1
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,5/12/2019,2
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,3/10/2019,2
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,4/21/2019,2


In [7]:
#Add "Category" column with "Food and Drink in San Diego" value
SD_events_df["Category"] = "Events"
SD_events_df.head()

,Event,Description,Neighborhood,Date,Cost,Category
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,1/20/2019,2,Events
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,3/16/2019,1,Events
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,5/12/2019,2,Events
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,3/10/2019,2,Events
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,4/21/2019,2,Events


In [8]:
#Find the datatypes
SD_events_df.dtypes

Event           object
Description     object
Neighborhood    object
Date            object
Cost             int64
Category        object
dtype: object

In [9]:
#Replace "Ongoing" value in Date column with blank to be able to change datatype of Date column 
#into a date type to be able to search and sort datafield by date
#Keepin in mind:  blank dates = NaT = "Ongoing"
SD_events_df = SD_events_df.replace({'Date':['Ongoing']},'')
SD_events_df['Date'] = pd.to_datetime(SD_events_df['Date'])

#Rename column "Event" to "Name"
SD_events_df = SD_events_df.rename(columns={"Event":"Name"})
SD_events_df.head()

,Name,Description,Neighborhood,Date,Cost,Category
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,2019-01-20,2,Events
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,2019-03-16,1,Events
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,2019-05-12,2,Events
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,2019-03-10,2,Events
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,2019-04-21,2,Events


In [10]:
#Clean File 1
#Sort Dataframe by name
SD_events_df = SD_events_df.sort_values("Name")
SD_events_df.head()

,Name,Description,Neighborhood,Date,Cost,Category
0,15th Annual Martin Luther King Jr Day Parade,(Not Provided),Downtown,2019-01-20,2,Events
1,18th Annual St Patricks Day Parade Festival,(Not Provided),Balboa Park,2019-03-16,1,Events
2,3rd Annual San Diego Bird Festival,(Not Provided),Mission Bay,2019-05-12,2,Events
3,4th Annual International Mariachi Festival,(Not Provided),South Bay,2019-03-10,2,Events
4,4th Annual Small Image Show,"Art Exhibition Featuring Works 10x10"" and Smaller",Downtown,2019-04-21,2,Events


In [11]:
#File 2: Attractions: TripAdvisor.csv

In [12]:
#Process File 2
trip_advisor_file = "Resources/TripAdvisor.csv"
trip_advisor_df = pd.read_csv(trip_advisor_file)
trip_advisor_df.head()

,Name,Address,Category,# of Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,$$$
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,$
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours Water Sports,1156,$$
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,$$
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,$


In [13]:
#Convert '$$' in Cost column to numeric
# get the length of the string of column in a dataframe
#http://www.datasciencemadesimple.com/get-string-length-column-dataframe-python-pandas/
#df['Quarters_length'] = df['Quarters'].apply(len)

trip_advisor_df.Cost = trip_advisor_df.Cost.apply(len)
trip_advisor_df.head()

,Name,Address,Category,# of Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,3
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,1
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours Water Sports,1156,2
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,2
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,1


In [14]:
#Replace "# of Reviews" column to "Reviews"
trip_advisor_df = trip_advisor_df.rename(columns={"# of Reviews":"Reviews"})
#Convert "Reviews" datatype to int 
trip_advisor_df['Reviews'] = trip_advisor_df['Reviews'].astype('int')
trip_advisor_df.head()

,Name,Address,Category,Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,3
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,1
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours Water Sports,1156,2
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,2
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,1


In [15]:
#get a list of the Category
category_list = trip_advisor_df["Category"].unique()
category_list = pd.DataFrame(category_list)
category_list.columns = ["Category List"]

#Sort Dataframe by Category List
category_list = category_list.sort_values("Category List")
category_list

,Category List
1,Boat Tours Water Sports
4,Classes Workshops
10,Concerts Shows
3,Food Drink
5,Fun Games
6,Nightlife
7,Outdoor Activities
9,Shopping
8,Spas Wellness
0,Things to do


In [16]:
trip_advisor_df = trip_advisor_df.replace({'Category':['Boat Tours  Water Sports']},\
                                                'Boat Tours & Water Sports')

trip_advisor_df = trip_advisor_df.replace({'Category':['Classes  Workshops']},\
                                                'Classes & Workshops')

trip_advisor_df = trip_advisor_df.replace({'Category':['Concerts  Shows']},\
                                                'Concerts & Shows')

trip_advisor_df = trip_advisor_df.replace({'Category':['Food  Drink']},\
                                                'Food & Drink')

trip_advisor_df = trip_advisor_df.replace({'Category':['Fun  Games']},\
                                                'Fun & Games')

trip_advisor_df = trip_advisor_df.replace({'Category':['Spas  Wellness']},\
                                                'Spas & Wellness')

trip_advisor_df.head()

,Name,Address,Category,Reviews,Cost
0,SeaWorld San Diego,"500 Sea World Drive, San Diego, CA 92109-7904",Things to do,9485,3
1,Mt. Soledad National Veterans Memorial,"6905 La Jolla Scenic Drive South, La Jolla, Sa...",Things to do,2240,1
2,Pacific Surf School,"4121 Mission Blvd, San Diego, CA 92109-5028",Boat Tours & Water Sports,1156,2
3,San Diego Fly Rides,"7444 Girard Ave, La Jolla, San Diego, CA 92037...",Tours,1113,2
4,Belmont Park,"3146 Mission Blvd, San Diego, CA 92109-7767",Things to do,764,1


In [17]:
#get new list of the Category to see changes
category_list = trip_advisor_df["Category"].unique()
category_list = pd.DataFrame(category_list)
category_list.columns = ["Category List"]

#Sort Dataframe by Category List
category_list = category_list.sort_values("Category List")
category_list

,Category List
1,Boat Tours & Water Sports
4,Classes & Workshops
10,Concerts & Shows
3,Food & Drink
5,Fun & Games
6,Nightlife
7,Outdoor Activities
9,Shopping
8,Spas & Wellness
0,Things to do


In [18]:
#Clean File 2
#Sort Dataframe by name
trip_advisor_df = trip_advisor_df.sort_values("Name")
trip_advisor_df.head()

,Name,Address,Category,Reviews,Cost
58,Action Sport Rentals,"3981 Mission Blvd, San Diego, CA 92109-6958",Boat Tours & Water Sports,18,2
79,Adventure Water Sports,"1710 W Mission Bay Dr Dana Hotel, San Diego, C...",Boat Tours & Water Sports,7,2
130,Adventure Water Sports,"2211 Pacific Beach Dr, San Diego, CA 92109-5626",Boat Tours & Water Sports,1,2
127,Affordable Jet Ski Tours,"4255 Mission Bay Dr, San Diego, CA 92109-5782",Boat Tours & Water Sports,1,2
77,Always Summer Surf School,"4904 Crystal Dr, San Diego, CA 92109-2062",Boat Tours & Water Sports,8,2


In [19]:
#File 3: Restaurants: Yelp_restaurants.csv

In [20]:
#Process File 3
yelp_file = "Resources/Yelp_restaurants.csv"
yelp_df = pd.read_csv(yelp_file)
yelp_df.head()

,Name,Address,Category,# of Reviews,Cost
0,Daybreak Island Grill,2296 Bacon St,Breakfast & Brunch,5111,$$
1,Bravo's Mexican Bistro & Cantina,5001 Newport Ave,Mexican,3054,$$
2,Caveman Pizza,1404 Vacation Rd,Pizza,2594,$$
3,Barefoot Bar & Grill,1404 Vacation Rd,American (New),1526,$$$
4,Saska's,3768 Mission Blvd,Steakhouses,1501,$$$$


In [21]:
#Convert '$$' in Cost column to numeric
# get the length of the string of column in a dataframe
#http://www.datasciencemadesimple.com/get-string-length-column-dataframe-python-pandas/
#df['Quarters_length'] = df['Quarters'].apply(len)

yelp_df.Cost = yelp_df.Cost.apply(len)
yelp_df.head()

,Name,Address,Category,# of Reviews,Cost
0,Daybreak Island Grill,2296 Bacon St,Breakfast & Brunch,5111,2
1,Bravo's Mexican Bistro & Cantina,5001 Newport Ave,Mexican,3054,2
2,Caveman Pizza,1404 Vacation Rd,Pizza,2594,2
3,Barefoot Bar & Grill,1404 Vacation Rd,American (New),1526,3
4,Saska's,3768 Mission Blvd,Steakhouses,1501,4


In [22]:
#add city and state in Addres to complete the address for geomapping 
yelp_df['Address'] = yelp_df.loc[:,["Address"]] + ", San Diego CA"
yelp_df.head()

,Name,Address,Category,# of Reviews,Cost
0,Daybreak Island Grill,"2296 Bacon St, San Diego CA",Breakfast & Brunch,5111,2
1,Bravo's Mexican Bistro & Cantina,"5001 Newport Ave, San Diego CA",Mexican,3054,2
2,Caveman Pizza,"1404 Vacation Rd, San Diego CA",Pizza,2594,2
3,Barefoot Bar & Grill,"1404 Vacation Rd, San Diego CA",American (New),1526,3
4,Saska's,"3768 Mission Blvd, San Diego CA",Steakhouses,1501,4


In [23]:
#Rename "# of Reviews" column to "Reviews"
yelp_df = yelp_df.rename(columns={"# of Reviews":"Reviews"})
#Convert "Reviews" datatype to int 
yelp_df['Reviews'] = yelp_df['Reviews'].astype('int')
yelp_df.head()

,Name,Address,Category,Reviews,Cost
0,Daybreak Island Grill,"2296 Bacon St, San Diego CA",Breakfast & Brunch,5111,2
1,Bravo's Mexican Bistro & Cantina,"5001 Newport Ave, San Diego CA",Mexican,3054,2
2,Caveman Pizza,"1404 Vacation Rd, San Diego CA",Pizza,2594,2
3,Barefoot Bar & Grill,"1404 Vacation Rd, San Diego CA",American (New),1526,3
4,Saska's,"3768 Mission Blvd, San Diego CA",Steakhouses,1501,4


In [24]:
#Rename "Category" column to "Description" column
yelp_df = yelp_df.rename(columns = {"Category": "Description"})

#Add "Category" column with "Food and Drink in San Diego" value
yelp_df["Category"] = "Food & Drink"
yelp_df.head()

,Name,Address,Description,Reviews,Cost,Category
0,Daybreak Island Grill,"2296 Bacon St, San Diego CA",Breakfast & Brunch,5111,2,Food & Drink
1,Bravo's Mexican Bistro & Cantina,"5001 Newport Ave, San Diego CA",Mexican,3054,2,Food & Drink
2,Caveman Pizza,"1404 Vacation Rd, San Diego CA",Pizza,2594,2,Food & Drink
3,Barefoot Bar & Grill,"1404 Vacation Rd, San Diego CA",American (New),1526,3,Food & Drink
4,Saska's,"3768 Mission Blvd, San Diego CA",Steakhouses,1501,4,Food & Drink


In [25]:
#Clean File 3 
yelp_df = yelp_df[["Name", "Description", "Address", "Category", "Reviews", "Cost"]]

#Sort Dataframe by name
yelp_df.sort_values("Name")
yelp_df.head()

,Name,Description,Address,Category,Reviews,Cost
0,Daybreak Island Grill,Breakfast & Brunch,"2296 Bacon St, San Diego CA",Food & Drink,5111,2
1,Bravo's Mexican Bistro & Cantina,Mexican,"5001 Newport Ave, San Diego CA",Food & Drink,3054,2
2,Caveman Pizza,Pizza,"1404 Vacation Rd, San Diego CA",Food & Drink,2594,2
3,Barefoot Bar & Grill,American (New),"1404 Vacation Rd, San Diego CA",Food & Drink,1526,3
4,Saska's,Steakhouses,"3768 Mission Blvd, San Diego CA",Food & Drink,1501,4


In [26]:
#Add latitude and longitude data to all files
yelp_df = yelp_df
locateAddresses(yelp_df)
print(error_count)
yelp_df.head()

0


,Name,Description,Address,Category,Reviews,Cost,Lat,Lng
0,Daybreak Island Grill,Breakfast & Brunch,"2296 Bacon St, San Diego CA",Food & Drink,5111,2,32.7532822,-117.2457722
1,Bravo's Mexican Bistro & Cantina,Mexican,"5001 Newport Ave, San Diego CA",Food & Drink,3054,2,32.7467997,-117.2513299
2,Caveman Pizza,Pizza,"1404 Vacation Rd, San Diego CA",Food & Drink,2594,2,32.7741456,-117.2404383
3,Barefoot Bar & Grill,American (New),"1404 Vacation Rd, San Diego CA",Food & Drink,1526,3,32.7741456,-117.2404383
4,Saska's,Steakhouses,"3768 Mission Blvd, San Diego CA",Food & Drink,1501,4,32.7839428,-117.2529375


In [27]:
#Convert query result to a csv file and use that file to drag and drop into a google map app to show the locations in the map.
yelp_df.to_csv('yelpLatLong.csv', index=False, header=False)

In [28]:
#Add latitude and longitude columns for gmapping using the locateAddresses function
locateAddresses(trip_advisor_df)
print(error_count)
trip_advisor_df.head()

0


,Name,Address,Category,Reviews,Cost,Lat,Lng
58,Action Sport Rentals,"3981 Mission Blvd, San Diego, CA 92109-6958",Boat Tours & Water Sports,18,2,32.7896789,-117.2531671
79,Adventure Water Sports,"1710 W Mission Bay Dr Dana Hotel, San Diego, C...",Boat Tours & Water Sports,7,2,32.7665947,-117.2374731
130,Adventure Water Sports,"2211 Pacific Beach Dr, San Diego, CA 92109-5626",Boat Tours & Water Sports,1,2,32.796177,-117.2245362
127,Affordable Jet Ski Tours,"4255 Mission Bay Dr, San Diego, CA 92109-5782",Boat Tours & Water Sports,1,2,32.8001516,-117.2131489
77,Always Summer Surf School,"4904 Crystal Dr, San Diego, CA 92109-2062",Boat Tours & Water Sports,8,2,32.8032044,-117.2599821


In [30]:
#Convert query result to a csv file and use that file to drag and drop into a google map app to show the locations in the map.
trip_advisor_df.to_csv('tirpadvisorLatLong.csv', index=False, header=False)

In [31]:
#Create database connection
connection_string = "root:Password@localhost/trip_planner_db?charset=utf8"
engine = create_engine(f'mysql://{connection_string}')

In [32]:
# Confirm tables
engine.table_names()

['attractions', 'events', 'restaurants']

In [33]:
#Delete all rows from the table and resets auto increment value for id to 1
engine.execute("TRUNCATE TABLE events;")
engine.execute("TRUNCATE TABLE attractions;")
engine.execute("TRUNCATE TABLE restaurants;")

In [34]:
#Load DataFrames into database
SD_events_df.to_sql(name='events', con=engine, if_exists='append', index=False)
trip_advisor_df.to_sql(name='attractions', con=engine, if_exists='append', index=False)
yelp_df.to_sql(name='restaurants', con=engine, if_exists='append', index=False)

In [35]:
#Test Queries

In [50]:
#Select from all tables all events, attractions and restaurants with cost range of 3 and 4.
expensive_activities_df = pd.read_sql_query('select * from (select * from events union select * from attractions union select * from restaurants) as a \
where a.Cost >= 3 AND a.Address IS NOT NULL order by a.Category;', con=engine)
expensive_activities_df.head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost,Lat,Lng
0,88,South Beach Bar & Grille,"5059 Newport Ave, San Diego CA",Seafood,Food & Drink,None,33,None,3,32.747381,-117.252323
1,4,Barefoot Bar & Grill,"1404 Vacation Rd, San Diego CA",American (New),Food & Drink,None,1526,None,3,32.774146,-117.240438
2,8,Tropics Bar & Grill,"1404 Vacation Rd, San Diego CA",American (New),Food & Drink,None,1142,None,3,32.774146,-117.240438
3,12,Draft,"3105 Ocean Front Walk, San Diego CA",American (Traditional),Food & Drink,None,842,None,4,32.769984,-117.252635
4,69,Cafe Bahia,"998 W Mission Bay Dr, San Diego CA",American (New),Food & Drink,None,120,None,3,32.772485,-117.246760


In [41]:
pd.read_sql_query('select * from (select * from events union select * from attractions union select * from restaurants) as a \
where a.Category = "Food & Drink" order by a.Name;', con=engine).head()

,id,Name,Address,Description,Category,Neighborhood,Reviews,Date,Cost,Lat,Lng
0,38,Arslan's Gyros,"3861 Mission Blvd, San Diego CA",Greek,Food & Drink,None,322,None,2,32.786091,-117.253179
1,11,Baja Winery Tours,"4629 Cass St # 2, San Diego, CA 92109-2805",None,Food & Drink,None,172,None,3,32.798907,-117.252065
2,87,Bao Beach,"3735 Mission Blvd, San Diego CA",Asian Fusion,Food & Drink,None,36,None,2,32.783325,-117.252416
3,4,Barefoot Bar & Grill,"1404 Vacation Rd, San Diego CA",American (New),Food & Drink,None,1526,None,3,32.774146,-117.240438
4,55,Bayside Landing,"3780 Ingraham St, San Diego CA",American (New),Food & Drink,None,236,None,3,32.788437,-117.237592
